# [Machine Learning with Core ML](https://www.packtpub.com/big-data-and-business-intelligence/machine-learning-core-ml)
**By:** Joshua Newnham (Author)  
**Publisher:** [Packt Publishing](https://www.packtpub.com/)

## Chapter 7 - Fast Neural Style Transfer 
In this notebook, we will look at 2 ways to optimize your model (here optimizing means reducing the models size). The first is to convert the precision your model uses for its weights from full-precision (32 bits) to half-precision (16 bits). The second uses a technique called quantization which, essentially, creates a lookup table to map a discrete set of values to each weight. 

**NB: To run locally;** create the environment from the *coreml27_2_environment.yml* file in this directory. Details of how to do this can be found [here](https://conda.io/docs/user-guide/tasks/manage-environments.html#creating-an-environment-from-an-environment-yml-file). 

In [ ]:
try:
    import coremltools
except:
    !pip install coremltools>=2.0b1   
    import coremltools 

In [ ]:
coreml_model = coremltools.models.MLModel('output/FastStyleTransferVanGoghStarryNight.mlmodel')

## Half-precision floating points 

We will use a utility function to convert a full precision (float) MLModel to a half precision MLModel (float16). Currently conversion for only neural network models is supported. If a pipeline model is passed in then all embedded neural network models embedded within will be converted. 

In [ ]:
fp16_coreml_model = coremltools.utils.convert_neural_network_weights_to_fp16(coreml_model)

In [ ]:
fp16_coreml_model.save('output/fp16_FastStyleTransferVanGoghStarryNight.mlmodel')

## Quantization

*Note; This feature is only available in coremltools 2.0b1 and onwards*. 

**Quantization** is the process of constraining an input from a continuous or otherwise large set of values (such as the real numbers) to a discrete set (such as the integers) - source [Wikipedia](https://en.wikipedia.org/wiki/Quantization). 

In [ ]:
from coremltools.models.neural_network import quantization_utils as quant_utils

*quant_utils.quantize_weights* is a utility function which performs the quantization; returning (if successful) the quantized model. The parameters include: 
- **full_precision_model:** MLModel Model which will be converted to half precision. Currently conversion for only neural network models is supported. If a pipeline model is passed in then all embedded neural network models embedded within will be converted.
- **nbits:** Number of bits per quantized weight. Only 8-bit and lower quantization is supported
- **quantization_mode:** Quantization algorithm used which can be *linear*, *linear_lut*, *kmeans_lut*, or *custom_lut*. Details (in brief) of each are provided below. 

Quantization modes include: 
- **linear:** Simple linear quantization with scale and bias
- **linear_lut:** Simple linear quantization represented as a lookup table
- **kmeans_lut:** LUT based quantization, where LUT is generated by K-Means clustering
- **custom_lut:** LUT quantization where LUT and quantized weight params are calculated using a custom function. If this mode is selected then a custom function must be passed in kwargs with key lut_function. The function must have input params (nbits, wp) where nbits is the number of quantization bits and wp is the list of weights for a given layer. The function should return two parameters (lut, qw) where lut is an array of length (2^nbits)containing LUT values and qw is the list of quantized weight parameters.

In [ ]:
lq8_coreml_model = quant_utils.quantize_weights(coreml_model, 8, 'linear')
lq4_coreml_model = quant_utils.quantize_weights(coreml_model, 4, 'linear')
km8_coreml_model = quant_utils.quantize_weights(coreml_model, 8, 'kmeans')
km4_coreml_model = quant_utils.quantize_weights(coreml_model, 4, 'kmeans')

In [ ]:
coremltools.models.MLModel(lq8_coreml_model) \
    .save('output/lq8_FastStyleTransferVanGoghStarryNight.mlmodel')
coremltools.models.MLModel(lq4_coreml_model) \
    .save('output/lq4_FastStyleTransferVanGoghStarryNight.mlmodel')
coremltools.models.MLModel(km8_coreml_model) \
    .save('output/km8_FastStyleTransferVanGoghStarryNight.mlmodel')
coremltools.models.MLModel(km4_coreml_model) \
    .save('output/km8_FastStyleTransferVanGoghStarryNight.mlmodel')